In [ ]:
!mkdir /content/MIMICData

In [ ]:
!unzip /content/TESTE.zip -d /content/MIMICData

Archive:  /content/TESTE.zip
   creating: /content/MIMICData/TESTE/
  inflating: /content/MIMICData/TESTE/fichier_1.txt  
  inflating: /content/MIMICData/TESTE/fichier_10.txt  
  inflating: /content/MIMICData/TESTE/fichier_2.txt  
  inflating: /content/MIMICData/TESTE/fichier_3.txt  
  inflating: /content/MIMICData/TESTE/fichier_4.txt  
  inflating: /content/MIMICData/TESTE/fichier_5.txt  
  inflating: /content/MIMICData/TESTE/fichier_6.txt  
  inflating: /content/MIMICData/TESTE/fichier_7.txt  
  inflating: /content/MIMICData/TESTE/fichier_8.txt  
  inflating: /content/MIMICData/TESTE/fichier_9.txt  


In [ ]:
!cat /content/MIMICData/TESTE/fichier_1.txt

L'intelligence artificielle (IA) est un « ensemble de théories et de techniques mises en œuvre en vue de réaliser des machines capables de simuler l'intelligence humaine »1.

Elle englobe donc un ensemble de concepts et de technologies, plus qu'une discipline autonome constituée2. Des instances, telle la CNIL, notant le peu de précision de la 
définition de l'IA, l'ont présentée comme « le grand mythe de notre temps »3.

Souvent classée dans le groupe des mathématiques et des sciences cognitives, elle fait appel à la neurobiologie computationnelle (particulièrement aux réseaux neuronaux) 
et à la logique mathématique (partie des mathématiques et de la philosophie). Elle utilise des méthodes de résolution de problèmes à forte complexité logique ou algorithmique.
 Par extension, elle comprend, dans le langage courant, les dispositifs imitant ou remplaçant l'homme dans certaines mises en œuvre de ses fonctions cognitives4.

Ses finalités et enjeux ainsi que son développement suscitent, de

In [ ]:
# Install the latest release of Haystack in your own environment 
#! pip install farm-haystack

!pip install --upgrade pip
!pip install git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab]


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 39.0 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 22.0.4
    Uninstalling pip-22.0.4:
      Successfully uninstalled pip-22.0.4
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/deepset-ai/haystack.git to /tmp/pip-install-ga_u5816/farm-haystack_6755acacc99e4864a64e1f89ccc209fd
  Running command git clone --filter=blob:none --quiet https://github.com/deepset-ai/haystack.git /tmp/pip-install-ga_u5816/farm-haystack_6755acacc99e4864a64e1f89ccc209fd
  Resolved https://github.com/deepset-ai/haystack.git to commit d2bba4935b2ccfa7ef875815a4a1bf98afcedbc1
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)

In [ ]:
from haystack.utils import clean_wiki_text, convert_files_to_docs, fetch_archive_from_http, print_answers
from haystack.nodes import FARMReader, TransformersReader

In [ ]:
# In Colab / No Docker environments: Start Elasticsearch from source
! wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
! tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
! chown -R daemon:daemon elasticsearch-7.9.2

import os
from subprocess import Popen, PIPE, STDOUT

es_server = Popen(
    ["elasticsearch-7.9.2/bin/elasticsearch"], stdout=PIPE, stderr=STDOUT, preexec_fn=lambda: os.setuid(1)  # as daemon
)
# wait until ES has started
! sleep 30

In [ ]:
from haystack.document_stores import ElasticsearchDocumentStore

document_store = ElasticsearchDocumentStore(host="localhost", username="", password="", index="document")

INFO:haystack.telemetry:Haystack sends anonymous usage data to understand the actual usage and steer dev efforts towards features that are most meaningful to users. You can opt-out at anytime by calling disable_telemetry() or by manually setting the environment variable  HAYSTACK_TELEMETRY_ENABLED as described for different operating systems on the documentation page. More information at https://docs.haystack.deepset.ai/docs/telemetry


In [ ]:
doc_dir = "/content/MIMICData"


# Convert files to dicts
# You can optionally supply a cleaning function that is applied to each doc (e.g. to remove footers)
# It must take a str as input, and return a str.
docs = convert_files_to_docs(dir_path=doc_dir, clean_func=clean_wiki_text, split_paragraphs=True)


INFO:haystack.utils.preprocessing:Converting /content/MIMICData/TESTE/fichier_8.txt
INFO:haystack.utils.preprocessing:Converting /content/MIMICData/TESTE/fichier_2.txt
INFO:haystack.utils.preprocessing:Converting /content/MIMICData/TESTE/fichier_9.txt
INFO:haystack.utils.preprocessing:Converting /content/MIMICData/TESTE/fichier_6.txt
INFO:haystack.utils.preprocessing:Converting /content/MIMICData/TESTE/fichier_10.txt
INFO:haystack.utils.preprocessing:Converting /content/MIMICData/TESTE/fichier_7.txt
INFO:haystack.utils.preprocessing:Converting /content/MIMICData/TESTE/fichier_5.txt
INFO:haystack.utils.preprocessing:Converting /content/MIMICData/TESTE/fichier_3.txt
INFO:haystack.utils.preprocessing:Converting /content/MIMICData/TESTE/fichier_4.txt
INFO:haystack.utils.preprocessing:Converting /content/MIMICData/TESTE/fichier_1.txt


In [ ]:
print(docs[:2])

# Now, let's write the dicts containing documents to our DB.
document_store.write_documents(docs)

[<Document: {'content': 'L’intelligence artificielle, ou IA, est entrain de changer le monde. Du domaine bancaire à l’entreprise, en passant par le secteur de l’agriculture ou encore la supply chain. Les applications de l‘IA sont nombreuses et variées ! Zoom sur l’intelligence artificielle en médecine à travers des exemples d’usage tel que l’aide au diagnostic.\nUsage de l’IA en médecine pour l’aide au diagnostic des cancers\nLa médecine l’a bien compris ! À l’image de l’hôpital parisien de la Pitié-Salpêtrière qui a rapidement adopté iBiopsy. Cette plateforme d’intelligence artificielle y révolutionne le traitement des pathologies, notamment le cancer du foie.\nDes méthodes d’imagerie médicale, de deep learning et big data analytics permettent de mieux extraire les biomarqueurs de la progression de la maladie. L’objectif est d’améliorer le diagnostic du patient – donc sa prise en charge.\n« On a une problématique, la détection du cancer. Et grâce à l’IA en médecine, on s’attache à y r

In [ ]:
from haystack.nodes import BM25Retriever

retriever = BM25Retriever(document_store=document_store)

In [ ]:
reader = FARMReader(model_name_or_path="etalab-ia/camembert-base-squadFR-fquad-piaf", use_gpu=True)

INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1
INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1


Downloading:   0%|          | 0.00/515 [00:00<?, ?B/s]

INFO:haystack.modeling.model.language_model: * LOADING MODEL: 'etalab-ia/camembert-base-squadFR-fquad-piaf' (Camembert)


Downloading:   0%|          | 0.00/443M [00:00<?, ?B/s]

INFO:haystack.modeling.model.language_model:Auto-detected model language: french
INFO:haystack.modeling.model.language_model:Loaded 'etalab-ia/camembert-base-squadFR-fquad-piaf' (Camembert model) from model hub.


Downloading:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/811k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/210 [00:00<?, ?B/s]

INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1


In [ ]:
from haystack.pipelines import ExtractiveQAPipeline

pipe = ExtractiveQAPipeline(reader, retriever)

In [ ]:
#question="What is artificial intelligence?"

In [ ]:
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import nltk

In [ ]:
from nltk.corpus import stopwords

In [ ]:
# Chargement des mots vides en français
nltk.download("stopwords")
stop_words = set(stopwords.words("french"))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# Fonction pour supprimer les mots vides
def remove_stop_words(text):
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return " ".join(filtered_words)

In [ ]:
# Texte à traiter
question="C'est quoi l'intelligence artificielle?"

In [ ]:
# Supprimer les mots vides
filtered_question = remove_stop_words(question)

print(filtered_question)

C'est quoi l'intelligence artificielle?


In [ ]:
# You can configure how many candidates the Reader and Retriever shall return
# The higher top_k_retriever, the better (but also the slower) your answers.
prediction = pipe.run(
    query=filtered_question, params={"Retriever": {"top_k": 10}, "Reader": {"top_k": 5}}
)

Inferencing Samples:   0%|          | 0/4 [00:00<?, ? Batches/s]

In [ ]:
from haystack.pipelines import DocumentSearchPipeline
pipeline = DocumentSearchPipeline(retriever)
query1 = 'est  intelligence artificielle? '
resultt = pipeline.run(query1, params={"Retriever": {"top_k": 10}})

In [ ]:
from haystack.utils import print_documents

print_documents(resultt, max_text_len=100, print_name=True, print_meta=True)


Query: est  intelligence artificielle? 

{   'content': 'ChatGPT, Midjourney… les IA sont plus que jamais dans '
               'l’actualité. Mais, quelle est la définition d’...',
    'meta': {'name': 'fichier_5.txt'},
    'name': 'fichier_5.txt'}

{   'content': "L'intelligence artificielle exploite les ordinateurs et les "
               'machines pour imiter les fonctions de ré...',
    'meta': {'name': 'fichier_2.txt'},
    'name': 'fichier_2.txt'}

{   'content': "L'intelligence artificielle (IA) est un « ensemble de théories "
               'et de techniques mises en œuvre en vu...',
    'meta': {'name': 'fichier_1.txt'},
    'name': 'fichier_1.txt'}

{   'content': "Définition de l'intelligence artificielle : L'intelligence "
               'artificielle (IA, ou AI en anglais pour A...',
    'meta': {'name': 'fichier_4.txt'},
    'name': 'fichier_4.txt'}

{   'content': 'Traduction automatique, reconnaissance d’image, reconnaissance '
               'faciale, assistants vocau

In [ ]:
from pprint import pprint

pprint(prediction['answers'])

[<Answer {'answer': 'IBM Cloud', 'type': 'extractive', 'score': 0.9873433113098145, 'context': "ur un montant de 400 millions de dollars.\nIntelligence artificielle et IBM Cloud\nIBM a été leader dans le développement des technologies basées sur l'", 'offsets_in_document': [{'start': 12868, 'end': 12877}], 'offsets_in_context': [{'start': 71, 'end': 80}], 'document_id': '9ff8f1535256ba93ac5d11b73b0ed2a4', 'meta': {'name': 'fichier_2.txt'}}>,
 <Answer {'answer': '(IA),', 'type': 'extractive', 'score': 0.9426077008247375, 'context': 'ergence de l’internet des objets (IoT) et de l’intelligence artificielle (IA), offrent maintenant aux patients la possibilité d’introduire des solutio', 'offsets_in_document': [{'start': 574, 'end': 579}], 'offsets_in_context': [{'start': 73, 'end': 78}], 'document_id': '9f3c3ec923144d93721e2165b835162d', 'meta': {'name': 'fichier_9.txt'}}>,
 <Answer {'answer': 'GPT-3.5', 'type': 'extractive', 'score': 0.907520592212677, 'context': 'sible de supprimer vos do

In [ ]:
from haystack.pipelines import DocumentSearchPipeline
pipeline = DocumentSearchPipeline(retriever)
query = 'c est quoi l intelligence artificielle? '
result = pipeline.run(query, params={"Retriever": {"top_k": 10}})


In [ ]:
from haystack.utils import print_documents

print_documents(result, max_text_len=100, print_name=True, print_meta=True)


Query: c est quoi l intelligence artificielle? 

{   'content': 'ChatGPT, Midjourney… les IA sont plus que jamais dans '
               'l’actualité. Mais, quelle est la définition d’...',
    'meta': {'name': 'fichier_5.txt'},
    'name': 'fichier_5.txt'}

{   'content': "Définition de l'intelligence artificielle : L'intelligence "
               'artificielle (IA, ou AI en anglais pour A...',
    'meta': {'name': 'fichier_4.txt'},
    'name': 'fichier_4.txt'}

{   'content': "L'intelligence artificielle (IA) est un « ensemble de théories "
               'et de techniques mises en œuvre en vu...',
    'meta': {'name': 'fichier_1.txt'},
    'name': 'fichier_1.txt'}

{   'content': "L'intelligence artificielle exploite les ordinateurs et les "
               'machines pour imiter les fonctions de ré...',
    'meta': {'name': 'fichier_2.txt'},
    'name': 'fichier_2.txt'}

{   'content': 'Traduction automatique, reconnaissance d’image, reconnaissance '
               'faciale, assistan

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
# Program to measure the similarity between
# two sentences using cosine similarity.
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# X = input("Enter first string: ").lower()
# Y = input("Enter second string: ").lower()
X =result1
Y =result

# tokenization
X_list = word_tokenize(X)
Y_list = word_tokenize(Y)

# sw contains the list of stopwords
sw = stopwords.words('french')
l1 =[];l2 =[]

# remove stop words from the string
X_set = {w for w in X_list if not w in sw}
Y_set = {w for w in Y_list if not w in sw}

# form a set containing keywords of both strings
rvector = X_set.union(Y_set)
for w in rvector:
	if w in X_set: l1.append(1) # create a vector
	else: l1.append(0)
	if w in Y_set: l2.append(1)
	else: l2.append(0)
c = 0

# cosine formula
for i in range(len(rvector)):
		c+= l1[i]*l2[i]
cosine = c / float((sum(l1)*sum(l2))**0.5)
print("similarity: ", cosine)

TypeError: ignored

In [ ]:
[x.to_dict() for x in result["documents"]]

[{'content': "Artificial intelligence (AI) is a wide-ranging branch of computer science concerned with building smart machines capable of performing tasks that typically require human intelligence. AI is an interdisciplinary science with multiple approaches, but advancements in machine learning and deep learning are creating a paradigm shift in virtually every sector of the tech industry. \nArtificial intelligence allows machines to model, and even improve upon, the capabilities of the human mind. \nFrom the development of self-driving cars to the proliferation of smart assistants like Siri and Alexa, AI is a growing part of everyday life. As a result, many tech companies across various industries are investing in artificially intelligent technologies.\nLess than a decade after helping the Allied forces win World War II by breaking the Nazi encryption machine Enigma, mathematician Alan Turing changed history a second time with a simple question: “Can machines think?” \nTuring’s 1950 pa